In [1]:
!pip install qiskit
!pip install qiskit_aer
!pip install pylatexenc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 92.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=f0559d80cba6d479d306243e4476e49f0f8ee3c43c10a3f47522d8fbb65038a2
  Stored in directory: /root/.cache/pip/wheels/b1/7a/33/9fdd892f784ed4afda62b685ae3703adf4c91aa0f524c28f03
Successfully built pylatexenc


In [2]:
import numpy as np
import random
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
from qiskit_aer import Aer, AerSimulator

In [3]:
def privacy_amp(key, bits_per_block=3, leakage_assumption=1):
    # set of 3 ... 0 xor 1 and 1 xor 2 .... make 2 bit
    if len(key) < bits_per_block:
        print(f"Key too short for privacy amplification ({len(key)} < {bits_per_block})")
        return key

    truncate = (len(key) // bits_per_block) * bits_per_block
    truncated_key = key[:truncate]

    compressed_key = []

    for i in range(0, truncate, bits_per_block):
        block = truncated_key[i:i+bits_per_block]


        compressed_key.append(block[0] ^ block[1])  # x xor y
        compressed_key.append(block[1] ^ block[2])  # y Xor z

    return compressed_key

In [4]:
n=100;


def encode(bits,basis):
  publickey=[]
  for i in range(n):
    qc=QuantumCircuit(1,1)
    if basis[i]==0: # z basis encoding 0=0 and 1=1
      if bits[i]==0:
        pass
      else:
        qc.x(0)
    else: # x basis encoding 0= |+> and 1= |->
      if bits[i]==0:
        qc.h(0)
      else:
        qc.x(0)
        qc.h(0)
    qc.barrier()
    publickey.append(qc)
  return publickey

def decode(msg,basis):
  measurements=[]
  for q in range(n):
    if basis[q]==0:
      msg[q].measure(0,0)
    if basis[q]==1:
      msg[q].h(0)
      msg[q].measure(0,0)
    counts=run_circuit(msg[q],shots=1)
    measured_bit=next(iter(counts))
    measurements.append(int(measured_bit))
  return measurements



def remove_garbage(a_basis, b_basis, bits):
    good_bits = []
    for q in range(n):
        if a_basis[q] == b_basis[q]:
            # if a==b then keep bits else reject
            good_bits.append(bits[q])
    return good_bits



def run_circuit(circuit, shots=1):
    simulator = AerSimulator()

    from qiskit import transpile
    transpiled_circuit = transpile(circuit, simulator)
    result = simulator.run(transpiled_circuit, shots=shots).result()
    counts = result.get_counts()

    return counts

In [5]:
key=np.random.randint(2,size=int((3*n)/2)) # <2 that means 0 and 1 only
print(key)

[0 0 0 1 1 0 1 1 1 1 0 1 1 0 1 0 0 1 0 1 0 1 0 1 1 1 0 1 0 1 0 0 1 1 1 0 1
 0 1 1 1 0 0 0 1 0 1 0 1 0 1 0 1 1 1 0 1 1 1 1 0 0 0 1 0 0 1 1 0 0 1 1 1 1
 0 1 0 0 1 1 1 0 1 1 1 1 0 1 0 1 0 1 0 0 1 1 0 1 0 0 1 1 1 1 0 1 1 0 1 1 1
 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 0 0 1 1 1 1 1 0 1 1 0 1 0 1 1 0 1 0 1 1 1 1
 1 0]


In [6]:
sharedkey=privacy_amp(key)
print(sharedkey)

[np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(1)

In [7]:
sender_basis = np.random.randint(2, size=n)
print(sender_basis) #0 for z basis and 1 for x basis

[0 1 0 1 0 1 0 0 1 1 0 1 1 0 1 0 1 0 0 0 1 1 0 1 1 0 1 1 1 1 0 1 1 1 0 1 1
 0 0 1 1 0 1 0 1 0 0 1 1 0 1 1 1 1 1 0 0 0 0 0 1 1 1 1 1 1 1 1 1 0 1 1 0 0
 0 1 0 1 1 0 0 1 1 0 0 0 1 1 0 0 0 0 1 0 1 0 1 1 0 1]


In [8]:
publickey=encode(sharedkey,sender_basis)
# display(publickey[0])
publickey[0].draw()

░ 
  q: ─░─
      ░ 
c: 1/═══

Reciever

In [9]:
def eve(quantum_states, eavesdropping_rate=0.3):
    # measure then try to decode then create a copy of the OG photon and send the copy
    intercepted_states = []

    #  eve basis
    eve_basis = np.random.randint(2, size=n)
    print(eve_basis)

    intercepted_indices = np.random.choice(n, size=int(n * eavesdropping_rate), replace=False)
    print(f"Eve intercepts {len(intercepted_indices)} qubits at positions:", intercepted_indices)

    for i in range(n):
        if i in intercepted_indices:
            eve_circuit = quantum_states[i].copy()

            # eve basis measurement
            if eve_basis[i] == 1:
                eve_circuit.h(0)

            eve_circuit.measure(0, 0)

            eve_counts = run_circuit(eve_circuit, shots=1)
            eve_result = int(next(iter(eve_counts)))

            # new qubit replacing og one
            new_circuit = QuantumCircuit(1, 1)

            # Eve prepares the state in the basis she used (not knowing Alice's original basis)
            if eve_basis[i] == 0:  # Z basis
                if eve_result == 1:
                    new_circuit.x(0)
            else:  # X basis
                if eve_result == 0:
                    new_circuit.h(0)
                else:
                    new_circuit.x(0)
                    new_circuit.h(0)

            new_circuit.barrier()
            intercepted_states.append(new_circuit)
        else:
            # Eve doesn't intercept this qubit
            intercepted_states.append(quantum_states[i].copy())

    return intercepted_states


intercepted_publickey=eve(publickey,0) # interception by eve creates error 0.5 means 50% interception

[1 0 0 1 0 0 1 1 1 0 1 0 0 1 0 1 1 1 0 1 1 0 0 1 1 0 0 0 1 0 1 1 0 1 1 0 1
 0 0 0 1 1 1 0 1 1 0 0 1 1 0 0 1 0 0 0 0 1 0 1 0 0 1 0 1 0 0 1 1 0 0 1 0 1
 0 1 0 0 0 1 0 0 0 1 1 0 1 1 1 0 0 1 1 1 0 0 1 1 1 1]
Eve intercepts 0 qubits at positions: []


In [10]:
def noise(quantum_states, noise_rate=0.05):

    noisy_states = []

    for i in range(len(quantum_states)):
        noisy_circuit = quantum_states[i].copy()

        if np.random.random() < noise_rate:
            # Random choice between bit flip (X), phase flip (Z), or both (Y)
            noise_type = np.random.randint(3)

            if noise_type == 0:  # Bit flip (X gate)
                noisy_circuit.x(0)
            elif noise_type == 1:  # Phase flip (Z gate)
                noisy_circuit.z(0)
            else:  # Bit+Phase flip (Y gate)
                noisy_circuit.y(0)

        noisy_states.append(noisy_circuit)

    return noisy_states

recieved_publickey=noise(intercepted_publickey,0) ## change this number for inducing noise

In [11]:


reciever_basis_guess = np.random.randint(2, size=n)

decodedkey=decode(recieved_publickey,reciever_basis_guess)
print(decodedkey)

[0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1]


In [12]:

reciever_key = remove_garbage(sender_basis, reciever_basis_guess, decodedkey)
print(reciever_key)
sender_key = remove_garbage(sender_basis, reciever_basis_guess, sharedkey)
int_list = [int(x) for x in sender_key]
print(int_list)


[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1]
[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1]


In [13]:
print("Keys match:", reciever_key == int_list)


match_percentage = len(reciever_key) / n * 100
print(f"Percentage of matching bases: {match_percentage:.2f}%")

# eavesdropping detection
def detect_eavesdropping(alice_bits, bob_bits, sample_size=None):
    if sample_size is None:
        sample_size = min(20, len(bob_bits)//4)
    check_positions = np.random.choice(range(len(bob_bits)), sample_size, replace=False)

    alice_sample = [alice_bits[i] for i in check_positions]
    bob_sample = [bob_bits[i] for i in check_positions]


    errors = sum(a != b for a, b in zip(alice_sample, bob_sample))
    error_rate = errors / sample_size


    remaining_positions = [i for i in range(len(bob_bits)) if i not in check_positions]
    alice_final = [alice_bits[i] for i in remaining_positions]
    bob_final = [bob_bits[i] for i in remaining_positions]

    return error_rate, alice_final, bob_final

error_rate, alice_final, bob_final = detect_eavesdropping(int_list, reciever_key)
print(f"Detected error rate: {error_rate:.2f}")
print(f"Final key length: {len(alice_final)}")

Keys match: True
Percentage of matching bases: 47.00%
Detected error rate: 0.00
Final key length: 36
